# FAMILIAS DE MALWARE
Alfredo Quezada - 191002
Randy Venegas - 

La primera parte de este laboratorio se basa en el analisis y creacion de un dataset que cuente con los datos obtenido por parte de los Malwares. 

### Parte 1:
##### Creacion del Dataset


In [9]:
# Lo primero que necesitamos es crear el Dataset, pero antes de eso necesitamos primero encontrar la informacion que
# llenara ese dataset, para eso se creo un codigo llamado "DatasetMaker.py", para crear el Dataset.

#Ahora, con el data set creado podemos hacer lo siguiente: 
import pandas as pd
db = pd.read_csv('dataset.csv')
print(db.shape)
db.sample(n=5).head()




(40, 4)


,Filename,PEHeader,Sections,Imports
5,RTC_7F85D7F628CE62D1D8F7B39D8940472,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'Name': 'UPX0', 'VirtualAddress': '0x1000'},...","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."
1,AAAz2E1B6940985A23E5639450F8391820655,"{'ImageBase': 268435456, 'SectionAlignment': 4...","[{'Name': 'UPX0', 'VirtualAddress': '0x1000'},...","[{'DLL': 'ADVAPI32.dll', 'Functions': ['RegSav..."
32,JH78C0A33A1B472A8C16123FD696A5CE5EBB,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'Name': 'UPX0', 'VirtualAddress': '0x1000'},...","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."
19,FTTR9EA3C16194CE354C244C1B74C46CD92E,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'Name': 'UPX0', 'VirtualAddress': '0x1000'},...","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."
31,33DE5067A433A6EC5C328067DC18EC37,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'Name': 'UPX0', 'VirtualAddress': '0x1000'},...","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."


Como podemos ver, aca tenemos la idea de lo que contiene el data set, realmente para los fines que necesitamos, consideramos que solo necesitamos, el PEHeader y los Imports, por lo que las secciones no lo vemos con mucha utilidad, por lo que como parte del **PREPROCESAMIENTO**, nos encargaremos de elimnar esta columna del dataset

In [10]:
# Eliminamos la que consideramos innecesaria 
db = db.drop(['Sections'], axis=1)
# y vemos lo que nos queda restante
db.sample(n=5).head()

,Filename,PEHeader,Imports
39,GFT4_7DDD3D72EAD03C7518F5D47650C8572,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'DLL': 'KERNEL32.dll', 'Functions': ['GetMod..."
10,L11_1415EB8519D13328091CC5C76A624E3D,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."
16,99A39866A657A10949FCB6D634BB30D5,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."
0,AL65_DB05DF0498B59B42A8E493CF3C10C578,"{'ImageBase': 268435456, 'SectionAlignment': 4...","[{'DLL': 'ADVAPI32.dll', 'Functions': ['RegSav..."
29,B07322743778B5868475DBE66EEDAC4F,"{'ImageBase': 4194304, 'SectionAlignment': 4096}","[{'DLL': 'KERNEL32.DLL', 'Functions': ['LoadLi..."


### Parte #2
##### Implementacion del Algoritmo